> This notebook is created and edited by Rohith Kulothungan (H00423559) for the coursework of Data Mining and Machine Leaning F21DL - 2023


## 1. Step 1 - Data Cleaning

We have chosen the gaming dataset to do text classification on the reviews provided into positive and negatives.

1. Understanding the data
    
  > Note: We have used the raw github url from the repository which contains the token that expires everyday. Kindly replace it before running the program. Thanks

In [1]:
import pandas as pd
import requests

def read_games_data():
    games = pd.read_csv('https://raw.githubusercontent.com/dmml-heriot-watt/group-coursework-ha/main/data/raw/games.csv?token=GHSAT0AAAAAACHSRUHF3CRZB7IHTCPEQABIZKE3NQQ')
    print(games)
    return games

games_data = read_games_data()

      Unnamed: 0                                    Title  Release Date  \
0              0                               Elden Ring  Feb 25, 2022   
1              1                                    Hades  Dec 10, 2019   
2              2  The Legend of Zelda: Breath of the Wild  Mar 03, 2017   
3              3                                Undertale  Sep 15, 2015   
4              4                            Hollow Knight  Feb 24, 2017   
...          ...                                      ...           ...   
1507        1507             Back to the Future: The Game  Dec 22, 2010   
1508        1508                        Team Sonic Racing  May 21, 2019   
1509        1509                           Dragon's Dogma  May 22, 2012   
1510        1510                          Baldur's Gate 3  Oct 06, 2020   
1511        1511                 The LEGO Movie Videogame  Feb 04, 2014   

                                                   Team  Rating Times Listed  \
0        ['Bandai N

2. Filter out only the required column which is the review and understanding the datastructure of the values

In [2]:
reviews = games_data['Reviews']
print(reviews[0])

["The first playthrough of elden ring is one of the best eperiences gaming can offer you but after youve explored everything in the open world and you've experienced all of the surprises you lose motivation to go exploring on repeat playthroughs which takes a lot away from the replayability which is a very important thing for from games imo.", 'a replay solidified my love for elden ring. so easily my favorite game of all time. actually beating malenia this time was also an amazing feeling. i just love being in this world man its the greatest of all time', 'The game is absolutely beautiful, with so much to do. The replayability is crazy. And it never gets old with it too.', 'Took everything great about the Soulsborne games and make it 100% better.', 'I play with my overlevelled friend every time and we still fail sometimes (he’s on NG6), insanely difficult game lol\n                     \n                     gorgeous graphics, animations, everything about this game is so beautiful. des

3. The reviews column is an array of strings which contains multiple reviews for a single game. 
4. Cleaning the data includes the following
    - Separating into single sentence of review
    - Removing single characters which doesn't add proper meaning
    - Removing special characters
    - Removing new lines & multiple spaces
    - Removing single worded reviews for better dataset

In [3]:
import re

def remove_single_characters(sentence):
    return re.sub(r'\W', ' ', str(sentence)) # removing single character

def remove_single_words(sentence):
    return re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence) # removing single words

def remove_special_characters(sentence):
    return re.sub(r'\^[a-zA-Z]\s+', ' ', sentence) # removing special characters

def remove_new_lines(sentence):
    return sentence.replace("\\n", ' ') # removing new lines

def replace_multiple_spaces(sentence):
    return re.sub(r"\s+", " ", sentence) # removing multiple spaces

cleaned_reviews = []

for review in reviews:
    texts = review[1:]
    texts = texts[:-1]
    if(len(texts) == 0):
        break
    line_present = True
    while(line_present):
        try:
            current_review = (texts[1:texts.find(texts[0], texts.find(texts) + 1)]).strip() # removing the quotes
            original_texts = current_review
            if(len(current_review) > 0):
                current_review = remove_single_characters(current_review)
                current_review = remove_single_words(current_review)
                current_review = remove_special_characters(current_review)
                if(len(current_review.strip()) == 0):
                    # if the current sentence length is zero skip and move to next sentence if present
                    remaining_texts = texts.split(original_texts, 1)
                    if (len(remaining_texts) > 1):
                        texts = remaining_texts[1][2:]
                        texts = texts.strip()
                        if (len(texts) == 0):
                            line_present = False
                    continue
                current_review = remove_new_lines(current_review)
                current_review = replace_multiple_spaces(current_review)
                cleaned_reviews.append(current_review)
                remaining_texts = texts.split(original_texts, 1)
                if(len(remaining_texts) > 1):
                    texts = remaining_texts[1][2:]
                    texts = texts.strip()
                    if(len(texts) == 0):
                        line_present = False
                else:
                    line_present = False
            else:
                line_present = False
        except:
            print(texts)

print(len(cleaned_reviews))


4149


In [4]:
df = pd.DataFrame(cleaned_reviews)
df.to_csv('cleaned_game_dataset.csv')

5. After cleaning the dataset we have 4149 rows of reviews that can be used for training the CNN. 